# 01 Reproducible Pipeline Design

Configs, run IDs, dataset hashes, and artifact layout.


## Table of Contents
- Configs
- Outputs
- Reproducibility


## Why This Notebook Matters
Model ops notebooks turn your work into reproducible runs with saved artifacts.
The goal is: someone else can run your pipeline and see the same metrics.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Not recording which dataset/config a model was trained on.
- Overwriting artifacts without run IDs.

## Matching Guide
- `docs/guides/05_model_ops/01_reproducible_pipeline_design.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/05_model_ops/01_reproducible_pipeline_design.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Understand run IDs, configs, and artifact layout in outputs/.


## Your Turn: Inspect scripts and configs


In [ ]:
# TODO: Open configs/recession.yaml and scripts/train_recession.py
# List what is configurable and what is hard-coded
...


## Your Turn: Run a pipeline


Run in terminal:
- python scripts/build_datasets.py --recession-config configs/recession.yaml --census-config configs/census.yaml
- python scripts/train_recession.py --config configs/recession.yaml
Then inspect outputs/<run_id>


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run one script end-to-end and confirm an artifact bundle exists.
# Example:
# - list outputs/ and pick the newest run_id
# - assert model.joblib and metrics.json exist
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Configs</summary>

```python
# Open configs/recession.yaml and configs/census.yaml and explain each field.
```

</details>

<details><summary>Solution: Outputs</summary>

```python
# Run:
#   python scripts/build_datasets.py --recession-config configs/recession.yaml --census-config configs/census.yaml
#   python scripts/train_recession.py --config configs/recession.yaml
# Then inspect outputs/<run_id>/
```

</details>

<details><summary>Solution: Reproducibility</summary>

```python
# Confirm run_metadata.json includes dataset hash and feature list.
```

</details>

